# TPS Jan 2022: Simple Catboost

In [ ]:
import sys
sys.path.append('../input/pycaret/pyod')
sys.path.append('../input/pycaret/pycaret')
from pycaret.regression import *

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
for df in [train, test]:
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['week']= df['date'].dt.week
    df['day'] = df.date.dt.day
    df['dayofweek'] = df.date.dt.dayofweek
    df['dayofyear'] = df['date'].dt.dayofyear
    df.drop(columns=['date'],inplace=True)  

In [ ]:
train.head()

In [ ]:
reg = setup(data = train,
            target = 'num_sold',
            train_size = 0.7, 
            normalize = True, 
            normalize_method = 'robust', 
            transform_target = True, 
            data_split_shuffle = False, 
            feature_interaction = True, 
            silent = True, 
            fold = 5, 
            n_jobs = -1) 

In [ ]:
def smape(y_true, y_pred):
    return np.mean( np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200 )

add_metric('SMAPE', 'SMAPE', smape, greater_is_better = False)

In [ ]:
model = create_model('catboost')

In [ ]:
pred = predict_model(model, data=test)
pred.head()

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
submission.num_sold = pred.Label
submission.to_csv('submission.csv', index = False)

In [ ]:
submission.head()